# Paper: Segmentation-Aware Convolutional Networks Using Local Attention Masks
> Adam W. Harley etl, ICCV 2017, Cited by 20 Until 01/29/2019,  
> See [code](https://github.com/aharley/segaware) and [pdf](http://openaccess.thecvf.com/content_ICCV_2017/papers/Harley_Segmentation-Aware_Convolutional_Networks_ICCV_2017_paper.pdf)

# Table of Contents
1. [Introduction](#introduction)
2. [Key Notation and Related Work](#key-notation-and-related-work)
3. [Technical approach](#technical-approach)

## Introduction <a name="introduction"></a>
There are three steps to creating segmentation-aware convolutional nets:
- (i) learn segmentation cues, 
- (ii) use the cues to create local foreground masks, and 
- (iii) use the masks together with convolution, to create foreground-focused convolution. 

This approach realizes each of these steps in a unified manner that is at once general (i.e., applicable to both discrete and continuous prediction tasks), differentiable (i.e., end-to-end trainable as a neural network), and fast (i.e., implemented as GPU-optimized variants of convolution).

## Key Notation and Related Work <a name="key-notation-and-related-work"></a>
- **metric learning**: the goal of metric learning is to produce features from which one can estimate the similarity between pixles or regions in the input.
- embeddings: refer to the featrues (that are invariant to rotation, scale, lighting, and partial occlusion, and the interior appearance details of objects) as embeddings, as they embed pixels into a space where the quality of correspondences can be measured as a distance.
- features that can capture the appearance of the foreground, while being invariant to changes in the background or occlusions.
- incorporates segmentation cues into CNNs, by sharpening or masking intermediate feature maps with the help of superpixels [12, 19]
- Concurrent work in language modelling [13] and image generation [40] has also emphasized the importance of **locally masked (or “gated”) convolutions**.
- mitigating the low-resolution and spatially-imprecise predictions of CNNs;
- atrous (aka dilated) convolution, for a wider field of view with the same number of parameters.
- "deconvolution" layers;
- To stack features from multiple resolutions near the end of the network, so that the final stages have access to both high-resolution (shallow) features and lowresolution (deep) features [22, 37, 14]. 
- the CRF approach has been integrated more closely with the CNN, by framing the CRF as a recurrent network, and chaining it to the backpropagation of the underlying CNN [61].

## Technical approach <a name="technical-approach"></a>
### Learning segmentation cues
Given an RGB image $I$, made up of pixels $p \in R^3$, enbedding function that maps (i.e., embeds) the pixels into a feature space where semantic similarity between pixels can be measured as a distance [11].
We can write the embedding function as $ f: R^3 \rightarrow R^D$, or more specifically, $f(p) = e$, where $e \in R^D$ is the embedding for pixel $p \in R^3$.

#### How to learn this function $f$ via CNN?
- For any two pixel indices $i$ and $j$, and corresponding embeddings $e_i$, $e_j$ and object class labels $I_i$, $I_j$, we can optimize the same-label pairs to have “near” embeddings, and the different-label pairs to have “far” embeddings. 
- Using $\alpha$ and $\beta$ to denote the “near” and “far” thresholds, respectively, we can define the pairwise loss as
$$ L_{i,j} = \begin{cases}
        max(|| e_i - e_j|| - \alpha, 0)  \text{    if  } I_i = I_j \\
        max(\beta - || e_i - e_j ||, 0)  \text{    if  } I_i \not = I_j
        \end{cases},  \text{ (1)} 
$$
- we use $\alpha = 0.5$, and $\beta = 2$. In practice, the specific values of $\alpha$ and $\beta$ are unimportant, so long as $\alpha \leq \beta$.
- Total loss is the summation across the image, with sampling pairs from a neighborhood around each pixel, as in $$ L = \sum_{i \in N} \sum_{j \in N_i} l_{i,j} \text{,  (2)}$$ where $j \in N_i$ iterates over the spatial neighbors of index $i$. For example, use $ 3 \times 3$ neighborhoods in practice.

#### Convert embedding distance to probabilities: **embeded mask**

- Convert the magnitudes into (unnormalized) probabilities indicating whether or not pixels $p_i$ and $p_j$ fall on the smae object, via the exponential distribution:
$$ m_{i,j} = \exp(- \lambda || e_i - e_j||)  \text{,  (3)}$$
- where $\lambda$ is a learnable parameter specifying the hardness of this decision, and the notation $m_{i,j}$ denotes that $i$ is the reference pixel, and $j$ is the neighbor being considered. 
- In other words, considering all indices $j \in N_i$ , $m_i$ represents a foreground-background segmentation mask, where the central pixel $i$ is defined as the foreground, i.e., $m_{i,i} = 1$.


### Segmentation-aware bilateral filtering
Given an input feature $x_i$, we can computer a segmentation-aware smoothed result, $y_i$, as follows:
$$ y_i = \frac{\sum_k x_{i-k} m_{i, i-k}}{\sum_k m_{i,i-k}} \text{,   (4)}$$

#### How this equation works?
- if $e_i = 0$ the equation yields the average filter;
- if $e_i = i$ the equation yields Gaussian smoothing;
- if $e_i = (i, p_i)$, where $p_i$ denotes the color vector at $i$, the equation yields [bilateral filtering](https://en.wikipedia.org/wiki/Bilateral_filter).

#### Note: In wiki, the bilateral filter is defined as
 
 $$ I^{\text{filtered}}(x)={\frac {1}{W_{p}}}\sum _{x_{i}\in \Omega }I(x_{i})f_{r}(\|I(x_{i})-I(x)\|)g_{s}(\|x_{i}-x\|),$$
where the normalization term $${\displaystyle W_{p}=\sum _{x_{i}\in \Omega }{f_{r}(\|I(x_{i})-I(x)\|)g_{s}(\|x_{i}-x\|)}} $$
ensures that the filter preserves image energy and

- $I^\text{filtered}$  is the filtered image;
- $I$ is the original input image to be filtered;
- $x$ are the coordinates of the current pixel to be filtered;
- $\Omega$  is the window centered in $x$;
- $f_r$ is the range kernel for smoothing differences in intensities (this function can be a Gaussian function);
- $g_{s}$ is the spatial kernel for smoothing differences in coordinates (this function can be a Gaussian function).

As mentioned above, the weight $W_{p}$ is assigned using the spatial closeness and the intensity difference. Consider a pixel located at $(i,j)$ that needs to be denoised in image using its neighbouring pixels and one of its neighbouring pixels is located at $(k, l)$. Then, the weight assigned for pixel $(k, l)$ to denoise the pixel $(i,j)$ is given by

$$ w(i,j,k,l)=\exp \left(-{\frac {(i-k)^{2}+(j-l)^{2}}{2\sigma _{d}^{2}}}-{\frac {\|I(i,j)-I(k,l)\|^{2}}{2\sigma _{r}^{2}}}\right),$$

where $\sigma_d$ and $\sigma_r$ are smoothing parameters, and $I(i, j)$ and $I(k, l)$ are the intensity of pixels $(i,j)$ and $(k, l)$ respectively.

After calculating the weights, normalize them:

$$I_{D}(i,j)={\frac {\sum _{k,l}I(k,l)w(i,j,k,l)}{\sum _{k,l}w(i,j,k,l)}},$$
where $I_{D}$ is the denoised intensity of pixel $(i,j)$.
